In [1]:
import re
import gc
import glob

import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('data/1行前との差分.xlsx')

df

,月,コード,金額1,金額2,diff
0,3,xxx,800,0,800
1,6,xxx,300,300,0
2,9,xxx,400,100,300
3,12,xxx,500,100,400
4,3,yyy,900,400,500
5,6,yyy,200,200,0
6,9,yyy,300,100,200
7,12,yyy,500,200,300
8,3,yyy,990,490,500
9,6,yyy,300,300,0


## 差分を1行上側に移動させたカラムを作成

In [3]:
df['diff(-1)'] = df.groupby('コード')['diff'].shift(-1)

df

,月,コード,金額1,金額2,diff,diff(-1)
0,3,xxx,800,0,800,0.0
1,6,xxx,300,300,0,300.0
2,9,xxx,400,100,300,400.0
3,12,xxx,500,100,400,NaN
4,3,yyy,900,400,500,0.0
5,6,yyy,200,200,0,200.0
6,9,yyy,300,100,200,300.0
7,12,yyy,500,200,300,500.0
8,3,yyy,990,490,500,0.0
9,6,yyy,300,300,0,300.0


## 1行上の値を比較する関数

In [4]:
def add_match_judge(df):
    
    
    for i in range(0, len(df)):
        # 0行目は比較対象がないため「1」を付与
        if i==0:
            df.loc[i,'judge_flag'] = '1'
        #1行目以降の処理
        else:
            # 月が6の場合はdiffの値が0であれば「1」を付与、そうでなければ「0」を付与
            if df.loc[i,'月']==6:
                if df.loc[i,'diff']==0:
                    df.loc[i,'judge_flag'] = '1'
                else:
                    df.loc[i,'judge_flag'] = '0'
            # 月が6以外場合は金額1の値とdiff(-1)の値が一致していれば「1」を付与、そうでなければ「0」を付与
            else:
                if df.loc[i,'金額1']==df.loc[i,'diff(-1)']:
                    df.loc[i,'judge_flag'] = '1'
                else:
                    df.loc[i,'judge_flag'] = '0'
                    
    return df
        

In [5]:
df1 = add_match_judge(df)

df1

,月,コード,金額1,金額2,diff,diff(-1),judge_flag
0,3,xxx,800,0,800,0.0,1
1,6,xxx,300,300,0,300.0,1
2,9,xxx,400,100,300,400.0,1
3,12,xxx,500,100,400,NaN,0
4,3,yyy,900,400,500,0.0,0
5,6,yyy,200,200,0,200.0,1
6,9,yyy,300,100,200,300.0,1
7,12,yyy,500,200,300,500.0,1
8,3,yyy,990,490,500,0.0,0
9,6,yyy,300,300,0,300.0,1


参考：https://teratail.com/questions/161719